In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
import numpy as np
import noise

from matplotlib import pyplot as plt
from matplotlib import colors as col
from mpl_toolkits.mplot3d import Axes3D

from ipywidgets import IntProgress, HTML, VBox
from IPython.display import display
import ipywidgets as wdg

import warnings
warnings.filterwarnings("ignore")

from plotly import graph_objs as plgo
from plotly import offline as ploff
ploff.init_notebook_mode(connected=False)

In [ ]:
def plot(x, y, land, water, title):
#     mask = ~water.astype(bool)
    mask = water < 0.00001
    layer = land + water
    layer[mask] = None
    ploff.iplot(
        plgo.Figure({
            'data': [
                plgo.Surface({
                    'name': 'land',
                    'x': x,
                    'y': y,
                    'z': land,
                    'colorscale': [(0.0, '#F9EBEA'), (1.0, '#641E16')],
                }),
                plgo.Surface({
                    'name': 'water',
                    'x': x,
                    'y': y,
                    'z': layer,
                    'colorscale': [(0.0, '#154360'), (1.0, '#85C1E9')],
                    'showscale': False,
                })
            ],
            'layout': plgo.Layout({
                'title': title,
                'width': 800,
                'height': 800,
                'scene': plgo.Scene({
                    'zaxis': plgo.ZAxis({
                        'range': [-1, 1.5]
                    })
                })
            }),
        })
    )

In [ ]:
def upper_plot(x, y, land, water, Timestep):
    mask = water[Timestep] < 0.00001
    layer = land + water[Timestep]
    layer[mask] = None
    land_cm = col.LinearSegmentedColormap.from_list(
        'land',
        [(110/255, 44/255, 0), (246/255, 221/255, 204/255)][::-1]
    )
    water_cm = col.LinearSegmentedColormap.from_list(
        'water',
        [(27/255, 79/255, 144/255), (214/255, 234/255, 248/255)]
    )
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(122)
    ax1.imshow(land, cmap=land_cm, vmin=np.min(land), vmax=np.max(land))
    ax1.imshow(layer, cmap=water_cm, vmin=np.min(land), vmax=np.max(land))
    ax2 = fig.add_subplot(121, projection='3d')
    ax2.plot_surface(x, y, land, cmap=land_cm)
    ax2.plot_surface(x, y, layer, cmap=water_cm)
    ax2.set_zlim(-1, 1.5)
    plt.show()

In [ ]:
np.random.seed(42)

# Flooding Model 2
---

by Francisco Casas & Vicente Lizana

## Momentum transfer model
---

Each cell has:
* A constant terrain level $t_{ij}$
* A water column level, $w_{ij}$
* A given momentum $(u_{ij},v_{ij})$: $\text{volume} \times \text{speed}$

Between the adjacent pairs of cells two metrics are computed:

* The water column difference (that allows discharge):
![Contact](figures/delta.png)
* The % of the water column over the other's terrain (one for each direction):
![Contact](figures/contact.png)

Water is transfered by 2 means:
* By discharge, due to water column difference.
* By its own speed (moment).

Applying the following rules:
* Of the water on the column difference is discharged each step to the receptor cell ($\alpha$).
* On a discharge, the receptor cell earns moment in the direction of the discharge ($\gamma$).
* Each cell also moves a fraction of its water due moment ($\beta$), as the water carries the moment, the same fraction of the moment is transfered.
    * This effect is multiplied by the column contact %.
    * Moment can be lost on the terrain.

<!-- No final transfer can be of more than $\frac{1}{4}$ of the cell's water, for the stability of the model. -->

In [ ]:
def momentum_transfer_automata(mom_x, mom_y, land, water,
        alpha, beta=1.0, gamma=0.2, disip=0.1):
    assert(land.shape == water.shape)
    assert(mom_x.shape == water.shape)
    assert(mom_y.shape == water.shape)
    levels = land + water

    # Get the % of contact of the water column
    water_div = water.copy()
    water_div[water_div==0] = 1.0
    #
    contact_e = np.zeros(water.shape)
    contact_e[:,:-1] = levels[:,:-1] - land[:,1:]
    contact_e = np.minimum(contact_e, water) / water_div
    #
    contact_w = np.zeros(water.shape)
    contact_w[:,1:] = levels[:,1:] - land[:,:-1]
    contact_w = np.minimum(contact_w, water) / water_div
    #
    contact_s = np.zeros(water.shape)
    contact_s[:-1,:] = levels[:-1,:] - land[1:,:]
    contact_s = np.minimum(contact_s, water) / water_div
    #
    contact_n = np.zeros(water.shape)
    contact_n[1:,:] = levels[1:,:] - land[:-1,:]
    contact_n = np.minimum(contact_n, water) / water_div

    # Get the water column difference
    discharge_x = levels[:,:-1] - levels[:,1:]
    discharge_x = np.minimum(water[:,:-1], discharge_x)
    discharge_x = np.maximum(-water[:,1:], discharge_x)
    #
    discharge_y = levels[:-1,:] - levels[1:,:]
    discharge_y = np.minimum(water[:-1,:], discharge_y)
    discharge_y = np.maximum(-water[1:,:], discharge_y)

    # 4 direction momentum:
    mom_e = np.maximum(mom_x,0)
    mom_w = np.maximum(-mom_x,0)
    mom_s = np.maximum(mom_y,0)
    mom_n = np.maximum(-mom_y,0)
    # Dissipate momentum due earth. #NOTE: Can be upgraded to bouncing!
    mom_e *= (1-disip) * (1-contact_e)
    mom_w *= (1-disip) * (1-contact_w)
    mom_s *= (1-disip) * (1-contact_s)
    mom_n *= (1-disip) * (1-contact_n)

    # Water movement due momentum:
    watert_e = mom_e * contact_e
    watert_w = mom_w * contact_w
    watert_s = mom_s * contact_s
    watert_n = mom_n * contact_n

    # Update waters:
    delta_x = discharge_x * alpha + watert_e[:,:-1] - watert_w[:,1:]
    delta_y = discharge_y * alpha + watert_n[:-1,:] - watert_s[1:,:]
    #
    nwater = water.copy()
    nwater[:,:-1] -= delta_x
    nwater[:,1:]  += delta_x
    nwater[:-1,:] -= delta_y
    nwater[1:,:]  += delta_y

    # Momentum transfer
    momt_e = mom_e * watert_e / water_div
    momt_w = mom_w * watert_w / water_div
    momt_s = mom_s * watert_s / water_div
    momt_n = mom_n * watert_n / water_div
    # Update momentums:
    mom_e -= momt_e
    mom_e[:,1:] += momt_e[:,:-1]
    mom_e[:,1:] += gamma * np.maximum(0, +delta_x)
    #
    mom_w -= momt_w
    mom_w[:,:-1] += momt_w[:,1:]
    mom_w[:,:-1] += gamma * np.maximum(0, -delta_x)
    #
    mom_s -= momt_s
    mom_s[1:,:] += momt_s[:-1,:]
    mom_s[1:,:] += gamma * np.maximum(0, +delta_y)
    #
    mom_n -= momt_n
    mom_n[:-1,:] += momt_n[1:,:]
    mom_n[:-1,:] += gamma * np.maximum(0, -delta_y)
    # NOTE: Here we can have anulation:
    nmom_x = mom_e - mom_w
    nmom_y = mom_s - mom_n

    return nmom_x, nmom_y, nwater

In [ ]:
base = np.linspace(0, 5, 100)
x, y = np.meshgrid(base, base)
# vectoriation of perlin noise
# final_func = np.vectorize(lambda x, y: noise.pnoise2(x, y, octaves=4))
# land = final_func(x, y)
land = np.zeros(x.shape)

# plot(x, y, land, np.zeros(land.shape), 'Terrain')

In [ ]:
water = np.zeros(land.shape)
water[45:56,45:56] = 1.0 - land[45:56,45:56]

In [ ]:
size = 1000
alpha = 0.2
gamma = 0.2
disip = 0.1

w = np.zeros((size, *land.shape))
mx = np.zeros((size, *land.shape))
my = np.zeros((size, *land.shape))
w[0,:,:] = water
for i in log_progress(range(0,size-1), every=1, name='Timesteps'):
    print(i,np.min(mx[i,:,:]),np.max(mx[i,:,:]))
    mx[i+1,:,:],my[i+1,:,:],w[i+1,:,:] = momentum_transfer_automata(
            mx[i,:,:],my[i,:,:],land,w[i,:,:],alpha,gamma,disip)

In [ ]:
play = wdg.Play(
#     interval=10,
    value=0,
    min=0,
    max=size-1,
    step=1,
    description="Press play",
    disabled=False
)
slider = wdg.IntSlider(
    min=0,
    max=size
)
wdg.jslink((play, 'value'), (slider, 'value'))
flooding = wdg.interactive(upper_plot, Timestep=slider, water=wdg.fixed(w), land=wdg.fixed(land), x=wdg.fixed(x), y=wdg.fixed(y))
display(wdg.VBox([play, flooding]))